# Deaths in nursing homes

In [1]:
# !/opt/anaconda3/bin/python -m pip install nb-black

In [2]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline
import json
import numpy as np
import altair as alt
import altair_latimes as lat

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
from datetime import datetime, date

In [5]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

In [62]:
df = pd.read_csv('../../california-coronavirus-data/cdph-adult-and-senior-care-totals.csv')

In [63]:
df.head()

,date,confirmed_cases,deaths,active_cases,source_url
0,2021-02-24,42023,3618,2440.0,https://www.cdss.ca.gov/Portals/9/Additional-R...
1,2021-02-22,41966,3594,2683.0,https://www.cdss.ca.gov/Portals/9/Additional-R...
2,2021-02-19,41863,3566,3587.0,https://www.cdss.ca.gov/Portals/9/Additional-R...
3,2021-02-18,41807,3543,3766.0,https://www.cdss.ca.gov/Portals/9/Additional-R...
4,2021-02-17,41756,3533,3928.0,https://www.cdss.ca.gov/Portals/9/Additional-R...


In [64]:
df_grouped = df.groupby(['date']).agg({'deaths':sum}).reset_index()

In [65]:
df_grouped['daily'] = df_grouped['deaths'].diff().fillna(df_grouped['deaths'])

In [66]:
df_grouped.loc[0,'daily'] = 6

In [67]:
df_grouped['avg'] = df_grouped['daily'].rolling(7).mean()

In [69]:
df_grouped.tail(20)

,date,deaths,daily,avg
221,2021-01-26,3068,41.0,66.428571
222,2021-01-27,3111,43.0,66.857143
223,2021-01-28,3144,33.0,66.714286
224,2021-01-29,3161,17.0,50.857143
225,2021-02-01,3226,65.0,54.428571
226,2021-02-02,3255,29.0,52.857143
227,2021-02-03,3287,32.0,37.142857
228,2021-02-04,3313,26.0,35.000000
229,2021-02-05,3346,33.0,33.571429
230,2021-02-08,3398,52.0,36.285714


In [76]:
deaths_chart = (
    alt.Chart(df_grouped)
    .mark_area()
    .encode(
        x=alt.X(
            "date:T",
            axis=alt.Axis(
                tickCount=4,
                format=("%b. %-d"),
                grid=False,
                tickColor="#ffffff",
                title="",
            ),
        ),
        y=alt.Y(
            "avg",
            title="",
            axis=alt.Axis(tickColor="#ffffff"),
        )
    )
    .properties(
        width=620,
        height=400,
        title="Nursing home deaths - seven-day avg.",
    )
    .configure_axis()
    .configure_legend(symbolType="stroke")
)

In [77]:
deaths_chart

alt.Chart(...)

---

In [82]:
df = pd.read_json('../../coronavirus-tracker/_data/nursing_homes/totals/timeseries.json')

In [84]:
df.head()

,date,confirmed_cases_skillednursing,deaths_skillednursing,active_cases_skillednursing,confirmed_cases_assistedliving,deaths_assistedliving,active_cases_assistedliving,confirmed_cases_nursinghomes,deaths_nursinghomes,active_cases_nursinghomes,confirmed_cases_statewide,deaths_statewide,confirmed_cases_nursinghomes_percent,deaths_nursinghomes_percent,new_confirmed_cases_skillednursing,new_deaths_skillednursing,new_confirmed_cases_assistedliving,new_deaths_assistedliving,new_confirmed_cases_nursinghomes,new_deaths_nursinghomes,new_confirmed_cases_skillednursing_seven_day_average,new_deaths_skillednursing_seven_day_average,new_confirmed_cases_assistedliving_seven_day_average,new_deaths_assistedliving_seven_day_average,new_confirmed_cases_nursinghomes_seven_day_average,new_deaths_nursinghomes_seven_day_average
0,2020-06-01,15420,1856,2723,2195,328,NaN,17615,2184,NaN,113006,4251,0.155877,0.513761,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-06-02,15495,1887,2723,2207,331,NaN,17702,2218,NaN,115310,4286,0.153517,0.517499,75,31,12,3,87,34,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-06-03,15728,1928,2713,2277,336,NaN,18005,2264,NaN,117687,4361,0.152991,0.519147,233,41,70,5,303,46,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-06-04,15907,1963,2775,2299,342,NaN,18206,2305,NaN,119807,4422,0.151961,0.521257,179,35,22,6,201,41,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-06-05,16132,1989,2767,2349,344,NaN,18481,2333,NaN,122901,4485,0.150373,0.520178,225,26,50,2,275,28,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
nursing_homes_chart = (
    alt.Chart(df)
    .mark_line()
    .encode(
        x=alt.X(
            "date:T",
            axis=alt.Axis(
                tickCount=4,
                format=("%b. %-d"),
                grid=False,
                tickColor="#ffffff",
                title="",
            ),
        ),
        y=alt.Y(
            "new_deaths_nursinghomes_seven_day_average",
            title="",
            axis=alt.Axis(tickColor="#ffffff"),
        )
    )
    .properties(
        width=620,
        height=400,
        title="Nursing home deaths - seven-day avg.",
    )
    .configure_axis()
    .configure_legend(symbolType="stroke")
)
nursing_homes_chart

alt.Chart(...)